# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import plotly.express as px
import statsmodels.formula.api as smf
import plotly.graph_objects as go
from scripts.python.routines.manifest import get_manifest
from scripts.python.routines.plot.save import save_figure
from scripts.python.routines.plot.layout import add_layout
from statsmodels.stats.multitest import multipletests
import plotly.io as pio
pio.kaleido.scope.mathjax = None
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=False)
from scipy.stats import mannwhitneyu, median_test, kruskal, wilcoxon, friedmanchisquare
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patheffects as path_effects
import random
import pathlib
from tqdm import tqdm
from src.utils.plot.bioinfokit import mhat, volcano
import gseapy as gp
import mygene
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA, TruncatedSVD
from sklearn.decomposition import MiniBatchDictionaryLearning, FastICA
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
from sklearn.manifold import MDS, Isomap, TSNE, LocallyLinearEmbedding
import upsetplot
from matplotlib_venn import venn2, venn2_circles
from itertools import chain
from sklearn.metrics import mean_absolute_error
from scripts.python.routines.plot.colorscales import get_continuous_color
from impyute.imputation.cs import fast_knn
import plotly
from scripts.python.routines.plot.p_value import add_p_value_annotation
from scripts.python.routines.sections import get_sections
from statannotations.Annotator import Annotator
import functools
import matplotlib.lines as mlines
import patchworklib as pw


def conjunction(conditions):
    return functools.reduce(np.logical_and, conditions)


def disjunction(conditions):
    return functools.reduce(np.logical_or, conditions)

# Load data

## Immunology

In [ ]:
path = f"D:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN"
path_save = f"{path}/special/060_EpiSImAge"
pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)

# Papers
df_imm_fimmu = pd.read_excel(f"{path}/data/immuno/models/SImAge/data.xlsx", index_col="sample_id")
df_imm_geroscience = pd.read_excel(f"{path}/data/immuno/models/IPAge/11357_2022_540_MOESM12_ESM.xlsx", index_col=0, skiprows=1)
df_epi_clinepi = pd.read_excel(f"{path}/data/GSE234461/samples.xlsx", index_col=0)

df_imm_all = pd.read_excel(f"{path}/data/immuno/df_samples(all_1052_121222)_proc(raw)_imp(fast_knn)_replace(quarter).xlsx", index_col=0)
df_ld_imm = df_imm_all['Subject ID'].value_counts().to_frame()
df_imm_all['Is longitudinal?'] = False
df_imm_all.loc[df_imm_all['Subject ID'].isin(df_ld_imm.index[df_ld_imm['Subject ID'] > 1].values), 'Is longitudinal?'] = True
df_imm_all.rename(columns={'Sample_Chronology': 'Time'}, inplace=True)
df_imm_all['Time'].replace({0: 'T0', 1: 'T1', 2: 'T2', 3: 'T3'}, inplace=True)
df_imm_ctrl = pd.read_excel(f"{path}/special/059_imm_data_selection/df_imm.xlsx", index_col=0)
df_imm_ctrl.rename(columns={'ids_fimmu': 'PMC10485620 ID'}, inplace=True)
df_imm_esrd = df_imm_all.loc[df_imm_all['Status'] == 'ESRD', :]
df_imm = pd.concat([df_imm_ctrl, df_imm_esrd], verify_integrity=True)
df_imm.loc[df_imm_fimmu.index.values, 'PMC10485620 ID'] = df_imm_fimmu.loc[df_imm_fimmu.index.values, 'index']
df_imm.loc[df_imm_geroscience.index.values, 'PMC9135940 ID'] = df_imm_geroscience.loc[df_imm_geroscience.index.values, 'ID_Origin']
df_imm.to_excel(f"{path_save}/df_imm.xlsx")

feats_imm = pd.read_excel(f"{path}/data/immuno/feats_con.xlsx", index_col=0).index.values
feats_imm_fimmu = pd.read_excel(f"{path}/data/immuno/models/SImAge/feats_con_top10.xlsx", index_col=0).index.values
feats_imm_slctd = pd.read_excel(f"{path}/special/059_imm_data_selection/feats_selected.xlsx", index_col=0).index.values

## Epigenetics

In [ ]:
df_epi_all = pd.read_excel(f"{path}/pheno.xlsx", index_col="index")
df_epi_all.index.name = "index"
df_epi_all.drop(["I64_old", "I1_duplicate"], inplace=True)
df_epi_all.rename(columns={'Subject_ID': 'Subject ID'}, inplace=True)
df_ld_epi = df_epi_all['Subject ID'].value_counts().to_frame()
df_epi_all['Is longitudinal?'] = False
df_epi_all.loc[df_epi_all['Subject ID'].isin(df_ld_epi.index[df_ld_epi['Subject ID'] > 1].values), 'Is longitudinal?'] = True
df_epi_all.rename(columns={'Sample_Chronology': 'Time'}, inplace=True)
df_epi_all['Time'].replace({0: 'T0', 1: 'T1', 2: 'T2', 3: 'T3'}, inplace=True)
df_epi = df_epi_all.loc[df_epi_all['Status'].isin(['Control', 'ESRD']), :]
df_epi.loc[df_epi_clinepi.index.values, 'GSM'] = df_epi_clinepi.loc[df_epi_clinepi.index.values, 'GSM']
df_epi.loc[df_epi_clinepi.index.values, 'PMC10699032 ID'] = df_epi_clinepi.loc[df_epi_clinepi.index.values, 'GSM']
df_epi.to_excel(f"{path_save}/df_epi.xlsx")

# Comparing

## Venn diagrams and sets

In [ ]:
pathlib.Path(f"{path_save}/imm_vs_epi").mkdir(parents=True, exist_ok=True)
sections = get_sections([set(df_imm.index), set(df_epi.index)])
for sec in sections:
    df_sec = pd.DataFrame(index=list(sections[sec]))
    df_sec.to_excel(f"{path_save}/imm_vs_epi/{sec}.xlsx", index_label='index')
fig, ax = plt.subplots()
venn = venn2(
    subsets=(set(df_imm.index), set(df_epi.index)),
    set_labels = ('Imm', 'Epi'),
    set_colors=('r', 'g'),
    alpha = 0.5
)
venn2_circles(subsets=(set(df_imm.index), set(df_epi.index)))
for text in venn.set_labels:
    text.set_fontsize(16)
for text in venn.subset_labels:
    text.set_fontsize(25)
plt.savefig(f"{path_save}/imm_vs_epi/venn.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path_save}/imm_vs_epi/venn.pdf", bbox_inches='tight')
plt.clf()

## Differences

In [ ]:
ids_cmn = df_epi.index.intersection(df_imm.index).values

feats_cmn = set(df_imm.columns.values).intersection(df_epi.columns.values)
feats_diff = {}
for f in feats_cmn:
    eq = df_imm.loc[ids_cmn, f].equals(df_epi.loc[ids_cmn, f])
    if not eq:
        ids_diff = ids_cmn[(df_imm.loc[ids_cmn, f] != df_epi.loc[ids_cmn, f]) == True]
        feats_diff[f] = ids_diff
        if f == "Age":
            print(f"{f}: {np.max(np.abs(df_imm.loc[ids_cmn, f].values - df_epi.loc[ids_cmn, f].values))}")
        else:
            print(f"{f}: {ids_diff}")

df_diff = pd.merge(df_imm.loc[ids_cmn, list(feats_diff.keys()) + ['Status', 'Region']], df_epi.loc[ids_cmn, list(feats_diff.keys())], left_index=True, right_index=True, suffixes=('_imm', '_epi'))
for f in feats_diff:
    df_diff[f"{f} Diff"] = False
    df_diff.loc[feats_diff[f], f"{f} Diff"] = True
df_diff.to_excel(f"{path_save}/imm_vs_epi/df_diff.xlsx")

# Merge data

In [ ]:
ids_cmn = df_epi.index.intersection(df_imm.index).values

feats_from_imm = list(feats_imm) + [
    'Subject ID', 'Time', 'Status', 'Sex', 'Age', 'Region', 'SImAge', 'SImAge acceleration', '|SImAge acceleration|', 'Dialysis (months)',
    'PMC10485620 ID', 'PMC9135940 ID'
]

feats_from_epi = [
    'index_origin', 'Aux', 'COVID', 'GSM', 'Residence', 'Nationality', 'Sample_Name', 'Sentrix_ID', 'Sentrix_Position',
]

df = pd.merge(df_imm.loc[ids_cmn, feats_from_imm], df_epi.loc[ids_cmn, feats_from_epi], left_index=True, right_index=True, suffixes=('_imm', '_epi'))

df_ltd = df['Subject ID'].value_counts().to_frame()
df['Is longitudinal?'] = False
df.loc[df['Subject ID'].isin(df_ltd.index[df_ltd['Subject ID'] > 1].values), 'Is longitudinal?'] = True
df.to_excel(f"{path_save}/df.xlsx")

# Create sample sheet for R

In [ ]:
df_r = df.loc[:, ['Subject ID', 'Sample_Name', 'Age', 'Sex', 'Status', 'Region', 'Sentrix_ID', 'Sentrix_Position']]
df_r['Tissue'] = 'Blood WB'
df_r.to_csv(f"{path_save}/sample_sheet.csv")